# Custom Tools

## 1. Way 1
** Using @tool decorator


This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/

In [ ]:
from langchain.tools import tool ,BaseTool
from langchain.pydantic_v1 import BaseModel, Field

**write a very simple function**

In [26]:
# write a very simple function
#decorate it with @tool
#place the docstring
@tool
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [27]:
type(fnc_multiply)


langchain_core.tools.StructuredTool

In [28]:
print(fnc_multiply.name)
print(fnc_multiply.description)
print(fnc_multiply.args)

fnc_multiply
fnc_multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


similar approach but this time we give it the tool a **separate name**

In [29]:
#similar approach but this time we give it the tool a **separate name**
@tool("multiply")
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [30]:
print(fnc_multiply.name)
print(fnc_multiply.description)
print(fnc_multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## 2. Way 2
** Using the Tool class - split the function & tool

In [31]:
from langchain.tools import Tool

def fnc_multiply(a: int, b: int) -> int:
    """Multiply 'a' times 'b'."""
    return a * b

multiply:Tool = Tool(
    name="multiply",
    func=fnc_multiply,
    description="Use this tool to Multiply 'a' times 'b'."
                )

In [32]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Use this tool to Multiply 'a' times 'b'.
{'tool_input': {'type': 'string'}}


## 3. Way 3
Subclass BaseTool

You can also explicitly define a custom tool by subclassing the BaseTool class. This provides maximal control over the tool definition, but is a bit more work.

In [ ]:

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Union


In [33]:
# a simple example !!

class Multiply(BaseTool):
    name="multiply"
    description="Use this tool to multiply two numbers"
    
    def _run(self,x:Union[int,float],y:Union[int,float])->float:
        return float(x * y)
    
    def _arun(self,x:Union[int,float],y:Union[int,float])->float:
        raise NotImplementedError("This tool does not support async")
    

In [35]:
# now more control !!
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Union

from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class MultiplyInput(BaseModel):
    x: int = Field(description="first number")
    y: int = Field(description="second number")

class Multiply(BaseTool):
    name="multiply"
    description="Use this tool to multiply two numbers"
    
    args_schema: Type[BaseModel] = MultiplyInput
    return_direct: bool = True
    
    def _run(self,x:Union[int,float],y:Union[int,float],run_manager:Optional[CallbackManagerForToolRun]=None)->float:
        return float(x * y)
    
    def _arun(self,x:Union[int,float],y:Union[int,float],run_manager:Optional[CallbackManagerForToolRun]=None)->float:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")


In [36]:
multiply = Multiply()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiply
Use this tool to multiply two numbers
{'x': {'title': 'X', 'description': 'first number', 'type': 'integer'}, 'y': {'title': 'Y', 'description': 'second number', 'type': 'integer'}}
True


## 4. Way 4
StructuredTool dataclass

You can also use a StructuredTool dataclass. This methods is a mix between the previous two. It's more convenient than inheriting from the BaseTool class, but provides more functionality than just using a decorator.

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

In [37]:
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

multiplyTool = StructuredTool.from_function(
    name="multiplyTool",
    func=fnc_multiply,
    description="Use this method to multiply two numbers."
)

In [38]:
print(multiplyTool.name)
print(multiplyTool.description)
print(multiplyTool.args)

multiplyTool
multiplyTool(a: int, b: int) -> int - Use this method to multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Comparision Direct LLM call vs. Agent Call !!!

In [39]:
from HelperAIKey import fnc_getOpenAIKey , fnc_getTavilyKey
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=fnc_getOpenAIKey(), verbose=True)  # Put your openAI key here 

In [40]:
llm.invoke('multiply 2 and 3')

AIMessage(content='2 x 3 = 6', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 13, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4c2bcf15-eea1-42f7-95d5-3f56c84f2b45-0')

In [41]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

tools = [multiplyTool]
llm_with_tools = llm.bind_tools([multiplyTool])

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "multiply 2 and 3", })



> Entering new AgentExecutor chain...

Invoking: `multiplyTool` with `{'a': 2, 'b': 3}`


6The result of multiplying 2 and 3 is 6.

> Finished chain.


{'input': 'multiply 2 and 3',
 'output': 'The result of multiplying 2 and 3 is 6.'}

In [42]:


llm.invoke('create a panda dataframe from the following [[1, 2, 3], [4, 5, 6], [7, 8, 9]] and parse the output to panda dataframe and return only the as final output')

AIMessage(content='import pandas as pd\n\ndata = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\ndf = pd.DataFrame(data)\ndf', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 55, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5417a33c-7d3f-4352-8133-5e4730296901-0')

In [43]:
lst = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
print(type(lst))

<class 'list'>


In [44]:
import pandas as pd

def fnc_pddffromlst(data:list) -> pd.DataFrame:
    """generate a Panda DataFrame from a list."""
    print('using fnc_pddffromlst')
    return pd.DataFrame(data)

fnc_pddffromlstTool = StructuredTool.from_function(
    name="fnc_pddffromlstTool",
    func=fnc_pddffromlst,
    description="Use this method to mgenerate a Panda DataFrame from a list."
)

In [45]:
import re
def fnc_pddffromdict(data) -> pd.DataFrame:
    """parse the output to panda dataframe and return as panda data frame."""
    print('using fnc_pddffromdict')
    print(data)
    #df_text = re.search(r'```(.*?)```', data['output'], re.DOTALL).group(1).strip()

    # Split the lines
    #lines = df_text.split('\n')

    # Convert the lines to a list of lists (skipping the header line)
    #data = [list(map(int, line.split())) for line in lines[1:]]

    #print(data)

    #pd.DataFrame(data)
    return pd.DataFrame(data)

fnc_pddffromdictTool = StructuredTool.from_function(
    name="fnc_pddffromdictTool",
    func=fnc_pddffromdict,
    description="Use this tool to parse the output to panda dataframe and return as panda data frame."
)

In [46]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

tools = [fnc_pddffromlstTool]
llm_with_tools = llm.bind_tools([fnc_pddffromlstTool])

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant, do not retrun the input tag, retrun only the final output"), 
    ("human", "{input}"),    
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)




_retVal = agent_executor.invoke({"input": "create a panda dataframe from the following [[1, 2, 3], [4, 5, 6], [7, 8, 9]] and parse the output to panda dataframe and return only the as final output" , })
print(_retVal)



> Entering new AgentExecutor chain...

Invoking: `fnc_pddffromlstTool` with `{'data': [[1, 2, 3], [4, 5, 6], [7, 8, 9]]}`


using fnc_pddffromlst
   0  1  2
0  1  2  3
1  4  5  6
2  7  8  9   0  1  2
0  1  2  3
1  4  5  6
2  7  8  9

> Finished chain.
{'input': 'create a panda dataframe from the following [[1, 2, 3], [4, 5, 6], [7, 8, 9]] and parse the output to panda dataframe and return only the as final output', 'output': '   0  1  2\n0  1  2  3\n1  4  5  6\n2  7  8  9'}


In [47]:
_retVal['output']

'   0  1  2\n0  1  2  3\n1  4  5  6\n2  7  8  9'

In [48]:
import io
def fnc_finalOutPut(data) -> pd.DataFrame:
    # Convert the string into a file-like object
    data_io = io.StringIO(data['output'])

# Read the data into a Pandas DataFrame
    df = pd.read_csv(data_io, sep='\\s+', index_col=0)

    #pd.DataFrame(data)
    return df

In [49]:
fnc_finalOutPut(_retVal)

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9
